In [95]:
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')


In [4]:
# Install a pip package in the current Jupyter Notebook for calulating MFCC Co-efficients
import sys
!{sys.executable} -m pip install python_speech_features

In [5]:
from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav

(rate,sig) = wav.read("dha_train/dha-spk-2.wav")
sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
X = mfcc(sig,rate,winstep=0.015)

In [41]:
N = 30
K = 3
#Initialising parameters randomly
pi = np.ones(K)/K
a = np.ones((K,K))
for r in range(K):
    a[r] = np.random.uniform(0,1,K)
    a[r] = a[r]/np.sum(a[r])
mu = np.random.randint(30,40,size=(K,13))
sigma = np.random.randint(10,50,size=(K,13,13))


#To make it a diagonal matrix

sigm = np.zeros((K,13,13))
for i in range(K):
    for j in range(13):
        for k in range(13):
            if(j == k):
                sigm[i][j][k] = sigma[i][j][k]
sigma = sigm*10
mu = mu/50

In [42]:
#For calculating PDF of Gaussian distribution
def gaussian(x,u,s):
    x.reshape(13,1)
    u.reshape(13,1)
    det = abs(np.linalg.det(s))**0.5
    inv = np.linalg.pinv(s)
    const = (2*np.pi)**(13/2)
    temp1 = const*det
    temp2 = np.exp((-0.5)*np.matmul((x-u).reshape(1,13),np.matmul(inv,(x-u).reshape(13,1))))
    #print(temp2)
    temp = temp2/temp1
    return temp[0][0]

In [43]:
#Calculating Alpha and Beta from parameters and also testing Alpha and Beta values for initial Parameters
def expectation_step(pi,a,mu,sigma):
    
    #parameters
    alpha = np.zeros((N,K))
    beta = np.zeros((N,K))
    beta[N-1] = np.ones(K)
    c = np.zeros(N)
    lhood = 0
    
    #Updating initial values of Alpha
    for k in range(K):
        alpha[0][k] = pi[k]*gaussian(X[0],mu[k],sigma[k])
    c[0] = sum(alpha[0])
    alpha[0] = alpha[0]/c[0]
    
    #Forward Algorithm to calculate alpha
    for n in range(1,N):
        for k in range(K):
            temp = 0
            for i in range(K):
                temp +=  alpha[n-1][i]*a[i][k]
            alpha[n][k] = temp*gaussian(X[n],mu[k],sigma[k])
            
        #Calculating Log-Likelihood for this set of parameters
        lhood = lhood + np.log(np.sum(alpha[n]))
        
        #Scaling Alpha
        c[n] = np.sum(alpha[n])
        alpha[n] = alpha[n]/c[n]        
    #Backward algorithm to calculate beta 
    for n in range(1,N):
        t = N-n-1
        s = 0
        for j in range(K):
            temp = 0
            for k in range(K):
                temp += beta[t+1][k]*a[j][k]*gaussian(X[t+1],mu[k],sigma[k])
            beta[t][j] = temp
        beta[t] = beta[t]/c[t+1]
    return alpha,beta,lhood,c
alpha,beta,lhood,c = expectation_step(pi,a,mu,sigma)
print(lhood)
print('Initial Alpha Values')
print(alpha)
print('Initial Beta Values')
print(beta)

-1512.8497236549351
Initial Alpha Values
[[0.09868696 0.1533338  0.74797924]
 [0.18615255 0.16916112 0.64468633]
 [0.20152816 0.19094365 0.60752819]
 [0.14871227 0.15005053 0.70123719]
 [0.17896417 0.15262869 0.66840715]
 [0.21451276 0.23571205 0.54977519]
 [0.15986115 0.19760603 0.64253282]
 [0.17480788 0.15882917 0.66636295]
 [0.16124655 0.16495771 0.67379574]
 [0.31359229 0.43628014 0.25012757]
 [0.16092133 0.36220508 0.47687359]
 [0.19793399 0.27838973 0.52367627]
 [0.31044111 0.37771473 0.31184417]
 [0.20883908 0.12748211 0.66367882]
 [0.2901534  0.15546359 0.55438301]
 [0.35747047 0.12856132 0.51396821]
 [0.31790469 0.13363677 0.54845854]
 [0.32545351 0.15003943 0.52450706]
 [0.25950046 0.13193652 0.60856302]
 [0.27226436 0.1391815  0.58855414]
 [0.22156099 0.15437717 0.62406183]
 [0.21971199 0.11715622 0.6631318 ]
 [0.27960128 0.1187578  0.60164093]
 [0.34538982 0.11470985 0.53990033]
 [0.24432645 0.09786522 0.65780833]
 [0.16394049 0.10420624 0.73185328]
 [0.21235455 0.0524733 

In [9]:
#Calculating Gamma and Sigma from Alpha and Beta
def gamma_sigma(alpha,beta,c):
    gamma = np.ones((N,K))
    #sig will be of dimensions N-1xKxK, but for simplicity of code we will be defining it as NxKxK
    sig = np.ones((N,K,K))
    #llid = sum(alpha[N-1]) 
    for n in range(N):
        for k in range(K):
            gamma[n][k] = alpha[n][k]*beta[n][k]  #/llid
    for n in range(1,N):
        for j in range(K):
            for k in range(K):
                sig[n][j][k] = c[n]*alpha[n-1][j]*gaussian(X[n],mu[k],sigma[k])*a[k][k]*beta[n][k] #/llid
    return gamma,sig 

In [10]:
#Updating the parameters of the model using gamma and sigma found
def maximization_step(gamma,sig,X):
    pi_new = np.ones(K)
    a_new = np.ones((K,K))
    mu_new = np.ones((K,13))
    sigma_new = np.ones((K,13,13))
    #Updating pi
    temp = 0
    for k in range(K):
        temp += gamma[0][k]
        pi_new[k] = gamma[0][k]
    pi_new = pi_new/temp
    #Updating A
    for k in range(K):
        temp2 = 0
        for i in range(K):
            temp1 = 0
            for n in range(1,N):
                temp1 += sig[n][k][i]
                temp2 += sig[n][k][i]
            a_new[k][i] = temp1
        a_new[k] = a_new[k]/temp2
    #Updating mu
    for k in range(K):
        temp3 = [0]*13
        temp4 = 0
        for n in range(N):
            temp3 += gamma[n][k]*X[n]
            temp4 += gamma[n][k]
        mu_new[k] = temp3/temp4
    #Updating sigma
    for k in range(K):
        temp5 = np.zeros((13,13))
        temp6 = 0
        for n in range(N):
            temp5 += gamma[n][k]* np.matmul((X[n]-mu_new[k]).reshape(13,1),(X[n]-mu_new[k]).reshape(1,13))
            temp6 += gamma[n][k]
        sigma_new[k] = temp5/temp6
    return pi_new,a_new,mu_new,sigma_new
#pi_new,a_new,mu_new,sigma_new = maximization_step(gamma,sig,X)

In [89]:
#HMM Model for training Phone-1 corresponding to Phone DHA
directory1 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\dha\\dha_train'
z1 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\dha\\dha_train\\'
def hmm_1():
    N = 30
    K = 3
    #Initialising parameters randomly
    pi = np.ones(K)/K
    a = np.ones((K,K))
    for r in range(K):
        a[r] = np.random.uniform(0,1,K)
        a[r] = a[r]/np.sum(a[r])
    mu = np.random.randint(30,40,size=(K,13))
    sigma = np.random.randint(10,50,size=(K,13,13))
    sigm = np.zeros((K,13,13))
    for i in range(K):
        for j in range(13):
            for k in range(13):
                if(j == k):
                    sigm[i][j][k] = sigma[i][j][k]
    #To make it a diagonal matrix
    sigma = sigm*10
    mu = mu/50
    prev_temp = None
    new_temp = 0
    loop = 1
    count = 0
    while(loop):
        count += 1
        for filename in os.listdir(directory1):
            (rate,sig) = wav.read(z1+filename)
            sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
            X = mfcc(sig,rate,winstep=0.015)
            X = X[0:30,:]
            alpha,beta,lhood,c = expectation_step(pi,a,mu,sigma)
            gamma,sig = gamma_sigma(alpha,beta,c)
            pi_new,a_new,mu_new,sigma_new = maximization_step(gamma,sig,X)
            pi,a,mu,sigma = pi_new,a_new,mu_new,sigma_new*1000
            new_temp += lhood
        new_temp = new_temp/30
        print('likelihood:',new_temp)
        if(prev_temp == None):
            prev_temp = new_temp
            new_temp = 0
        else:
            print(np.abs((np.abs(prev_temp) - np.abs(new_temp))))
            if(np.abs((np.abs(prev_temp) - np.abs(new_temp))) < 0.001 ):
                print('Done')
                loop = 0
            prev_temp = new_temp
            new_temp = 0
    return pi,a,mu,sigma           
pi_1,a_1,mu_1,sigma_1 = hmm_1()

D:\PROGRAMMING\ANACONDA\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in short_scalars


likelihood: -2345.9094280532136
likelihood: -2370.3055734015315
24.396145348317987
likelihood: -2370.3057197972403
0.00014639570872532204
Done


In [90]:
#HMM Model for training Phone-2 corresponding to Phone KHO
directory2 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\kho\\kho_train'
z2 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\kho\\kho_train\\'
def hmm_2():
    N = 30
    K = 3
    #Initialising parameters randomly
    pi = np.ones(K)/K
    a = np.ones((K,K))
    for r in range(K):
        a[r] = np.random.uniform(0,1,K)
        a[r] = a[r]/np.sum(a[r])
    mu = np.random.randint(30,40,size=(K,13))
    sigma = np.random.randint(10,50,size=(K,13,13))
    sigm = np.zeros((K,13,13))
    for i in range(K):
        for j in range(13):
            for k in range(13):
                if(j == k):
                    sigm[i][j][k] = sigma[i][j][k]
    #To make it a diagonal matrix
    sigma = sigm*10
    mu = mu/50
    prev_temp = None
    new_temp = 0
    loop = 1
    count = 0
    while(loop):
        count += 1
        for filename in os.listdir(directory2):
            (rate,sig) = wav.read(z2+filename)
            sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
            X = mfcc(sig,rate,winstep=0.015)
            X = X[0:30,:]
            alpha,beta,lhood,c = expectation_step(pi,a,mu,sigma)
            gamma,sig = gamma_sigma(alpha,beta,c)
            pi_new,a_new,mu_new,sigma_new = maximization_step(gamma,sig,X)
            pi,a,mu,sigma = pi_new,a_new,mu_new,sigma_new*1000
            new_temp += lhood
        new_temp = new_temp/30
        print('likelihood:',new_temp)
        if(prev_temp == None):
            prev_temp = new_temp
            new_temp = 0
        else:
            print(np.abs((np.abs(prev_temp) - np.abs(new_temp))))
            if(np.abs((np.abs(prev_temp) - np.abs(new_temp))) < 0.001 ):
                print('Done')
                loop = 0
            prev_temp = new_temp
            new_temp = 0
    return pi,a,mu,sigma           
pi_2,a_2,mu_2,sigma_2 = hmm_2()

D:\PROGRAMMING\ANACONDA\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in short_scalars


likelihood: -2342.4155041436447
likelihood: -2362.351796673491
19.93629252984647
likelihood: -2362.1759087932032
0.17588788028797353
likelihood: -2362.1417698172786
0.034138975924633996
likelihood: -2362.1353009421086
0.006468875169957755
likelihood: -2362.134078059913
0.0012228821956341562
likelihood: -2362.1338460746747
0.00023198523831524653
Done


In [105]:
def classifier(pi_1,a_1,mu_1,sigma_1,pi_2,a_2,mu_2,sigma_2,path):
    (rate,sig) = wav.read(path)
    sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
    X = mfcc(sig,rate,winstep=0.015)
    print(np.shape(X))
    #parameters
    alpha_1 = np.zeros((N,K))
    c_1 = np.zeros(N)
    lhood_1 = 0
    
    for k in range(K):
        alpha_1[0][k] = pi_1[k]*gaussian(X[0],mu_1[k],sigma_1[k])
    c_1[0] = sum(alpha_1[0])
    alpha_1[0] = alpha_1[0]/c_1[0]
    
    #Forward Algorithm to calculate alpha
    for n in range(1,N):
        for k in range(K):
            temp = 0
            for i in range(K):
                temp +=  alpha_1[n-1][i]*a_1[i][k]
            alpha_1[n][k] = temp*gaussian(X[n],mu_1[k],sigma_1[k])   
        #Calculating Log-Likelihood for this set of parameters
        lhood_1 = lhood_1 + np.sum(alpha_1[n])
    lhood_1 = np.log(lhood_1)
    #parameters
    alpha_2 = np.zeros((N,K))
    c_2 = np.zeros(N)
    lhood_2 = 0
    
    for k in range(K):
        alpha_2[0][k] = pi_2[k]*gaussian(X[0],mu_2[k],sigma_2[k])
    c_2[0] = sum(alpha_2[0])
    alpha_2[0] = alpha_2[0]/c_2[0]
    
    #Forward Algorithm to calculate alpha
    for n in range(1,N):
        for k in range(K):
            temp = 0
            for i in range(K):
                temp +=  alpha_2[n-1][i]*a_2[i][k]
            alpha_2[n][k] = temp*gaussian(X[n],mu_2[k],sigma_2[k])
        #Calculating Log-Likelihood for this set of parameters
        lhood_2 = lhood_2 + np.sum(alpha_2[n])
    lhood_2 = np.log(lhood_2)
    print(lhood_1,lhood_2)

In [106]:
classifier(pi_1,a_1,mu_1,sigma_1,pi_2,a_2,mu_2,sigma_2,'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\kho\\kho_test\\kho-spk-1.wav'
)
print('Test Sample belonging to KHO phone is classified as KHO Phone as its Log Likelihood is higher but the margin is very close for K=3')

(37, 13)
-73.90221195289087 -72.88616699204088
Test Sample belonging to KHO phone is classified as KHO Phone as its Log Likelihood is higher but the margin is very close for K=3


In [121]:
#Classifying for higher K values ------ K=5
#HMM Model for training Phone-1 corresponding to Phone DHA
directory1 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\dha\\dha_train'
z1 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\dha\\dha_train\\'
def hmm_3():
    N = 30
    K = 5
    #Initialising parameters randomly
    pi = np.ones(K)/K
    a = np.ones((K,K))
    for r in range(K):
        a[r] = np.random.uniform(0,1,K)
        a[r] = a[r]/np.sum(a[r])
    mu = np.random.randint(30,40,size=(K,13))
    sigma = np.random.randint(10,50,size=(K,13,13))
    sigm = np.zeros((K,13,13))
    for i in range(K):
        for j in range(13):
            for k in range(13):
                if(j == k):
                    sigm[i][j][k] = sigma[i][j][k]
    #To make it a diagonal matrix
    sigma = sigm*10
    mu = mu/50
    prev_temp = None
    new_temp = 0
    loop = 1
    count = 0
    while(loop):
        count += 1
        for filename in os.listdir(directory1):
            (rate,sig) = wav.read(z1+filename)
            sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
            X = mfcc(sig,rate,winstep=0.015)
            X = X[0:30,:]
            alpha,beta,lhood,c = expectation_step(pi,a,mu,sigma)
            gamma,sig = gamma_sigma(alpha,beta,c)
            pi_new,a_new,mu_new,sigma_new = maximization_step(gamma,sig,X)
            pi,a,mu,sigma = pi_new,a_new,mu_new,sigma_new*1000
            new_temp += lhood
        new_temp = new_temp/30
        print('likelihood:',new_temp)
        if(prev_temp == None):
            prev_temp = new_temp
            new_temp = 0
        else:
            print(np.abs((np.abs(prev_temp) - np.abs(new_temp))))
            if(np.abs((np.abs(prev_temp) - np.abs(new_temp))) < 0.001 ):
                print('Done')
                loop = 0
            prev_temp = new_temp
            new_temp = 0
    return pi,a,mu,sigma           
pi_3,a_3,mu_3,sigma_3 = hmm_3()

likelihood: -2341.343875420025
likelihood: -2356.9741575854205
15.630282165395329
likelihood: -2356.2882365046053
0.6859210808152056
likelihood: -2356.2781117298405
0.010124774764790345
likelihood: -2356.2779923339654
0.00011939587511733407
Done


In [100]:
#HMM Model for training Phone-2 corresponding to Phone KHO
directory2 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\kho\\kho_train'
z2 = 'C:\\Users\\SAI ASHOK\\OneDrive\\Desktop\\kho\\kho_train\\'
def hmm_4():
    N = 30
    K = 5
    #Initialising parameters randomly
    pi = np.ones(K)/K
    a = np.ones((K,K))
    for r in range(K):
        a[r] = np.random.uniform(0,1,K)
        a[r] = a[r]/np.sum(a[r])
    mu = np.random.randint(30,40,size=(K,13))
    sigma = np.random.randint(10,50,size=(K,13,13))
    sigm = np.zeros((K,13,13))
    for i in range(K):
        for j in range(13):
            for k in range(13):
                if(j == k):
                    sigm[i][j][k] = sigma[i][j][k]
    #To make it a diagonal matrix
    sigma = sigm*10
    mu = mu/50
    prev_temp = None
    new_temp = 0
    loop = 1
    count = 0
    while(loop):
        count += 1
        for filename in os.listdir(directory2):
            (rate,sig) = wav.read(z2+filename)
            sig = (sig -np.min(sig))/(np.max(sig)-np.min(sig))
            X = mfcc(sig,rate,winstep=0.015)
            X = X[0:30,:]
            alpha,beta,lhood,c = expectation_step(pi,a,mu,sigma)
            gamma,sig = gamma_sigma(alpha,beta,c)
            pi_new,a_new,mu_new,sigma_new = maximization_step(gamma,sig,X)
            pi,a,mu,sigma = pi_new,a_new,mu_new,sigma_new*1000
            new_temp += lhood
        new_temp = new_temp/30
        print('likelihood:',new_temp)
        if(prev_temp == None):
            prev_temp = new_temp
            new_temp = 0
        else:
            print(np.abs((np.abs(prev_temp) - np.abs(new_temp))))
            if(np.abs((np.abs(prev_temp) - np.abs(new_temp))) < 0.001 ):
                print('Done')
                loop = 0
            prev_temp = new_temp
            new_temp = 0
    print(K)
    return pi,a,mu,sigma           
pi_4,a_4,mu_4,sigma_4 = hmm_4()

likelihood: -2353.6192697230126
likelihood: -2369.5438205976793
15.924550874666693
likelihood: -2365.268539662916
4.275280934763487
likelihood: -2363.079075660537
2.189464002378827
likelihood: -2362.3497102103775
0.7293654501595483
likelihood: -2362.1772258792403
0.1724843311371842
likelihood: -2362.142249007213
0.034976872027073114
likelihood: -2362.135431856478
0.00681715073505984
likelihood: -2362.134110208148
0.001321648330304015
likelihood: -2362.1338535252785
0.00025668286934887874
Done
5
